## Binary structure classification used in tree building

1. Create train and test sets; Save negative samples of file ``filename.rs3`` as `filename.neg`
2. Train models, save the best one.

Output:
 - ``data/*.neg``
 - ``models/structure_predictor/*``

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import pandas as pd
import pickle
from utils.file_reading import read_edus, read_gold, read_negative, read_annotation

### Make a directory

In [3]:
MODEL_PATH = 'models/structure_predictor_lstm'
! mkdir $MODEL_PATH

TRAIN_FILE_PATH = os.path.join(MODEL_PATH, 'structure_cf_train.tsv')
TEST_FILE_PATH = os.path.join(MODEL_PATH, 'structure_cf_test.tsv')

mkdir: cannot create directory ‘models/structure_predictor_lstm’: File exists


###  Generate train/test files

#### Make train set 

In [4]:
from utils.train_test_split import split_data

train, test = split_data('./data')

news in train: 0.3886792452830189,	in test: 0.3939393939393939
ling in train: 0.1509433962264151,	in test: 0.15151515151515152
comp in train: 0.1471698113207547,	in test: 0.15151515151515152
blog in train: 0.3132075471698113,	in test: 0.3181818181818182


In [5]:
MAX_LEN = 200

In [6]:
from tqdm.autonotebook import tqdm


random_state = 45
train_samples = []

for file in tqdm(train):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['relation'] = 1
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    sample = gold[['relation', 'snippet_x', 'snippet_y']]
    negative = read_negative(file.replace('.edus', ''), features=True)
    negative['relation'] = 0
    negative['len_x'] = negative.tokens_x.map(len)
    negative = negative[negative.len_x < MAX_LEN]
    negative['len_y'] = negative.tokens_y.map(len)
    negative = negative[negative.len_y < MAX_LEN]
    negative['snippet_x'] = negative.tokens_x.map(lambda row: ' '.join(row))
    negative['snippet_y'] = negative.tokens_y.map(lambda row: ' '.join(row))
    sample = pd.concat([sample, negative[['relation', 'snippet_x', 'snippet_y']]])
    sample = sample.sort_values(['relation'], ascending=True).drop_duplicates(['snippet_x', 'snippet_y'], keep='last')    
    train_samples.append(sample)

train_samples = pd.concat(train_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
train_samples.reset_index(level=0, inplace=True)

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [7]:
train_samples.relation.value_counts()

0    66420
1    25728
Name: relation, dtype: int64

In [8]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].head()

,relation,snippet_x,snippet_y,index
0,0,основанного прежде всего на логических операци...,"Во - вторых , сомневаться в истинности языка н...",0
1,1,"Восстанавливаемыми называют объекты , допускаю...",Невосстанавливаемые же в процессе выполнения с...,1
2,0,Что может - отложить и подвигать окно качества .,В рамках этой модели первое что должен сделать...,2
3,0,"нередко отличается безжалостностью ,",так как в его ментальном состоянии происходит ...,3
4,0,"Между тем , представляется , что среди концепт...","С этой точки зрения интересно решение , найден...",4


In [9]:
train_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TRAIN_FILE_PATH, sep='\t', header=False, index=False)

#### Make test set

In [10]:
random_state = 45
test_samples = []

for file in tqdm(test):
    gold = read_gold(file.replace('.edus', ''), features=True)
    gold['relation'] = 1
    gold['len_x'] = gold.tokens_x.map(len)
    gold = gold[gold.len_x < MAX_LEN]
    gold['len_y'] = gold.tokens_y.map(len)
    gold = gold[gold.len_y < MAX_LEN]
    gold['snippet_x'] = gold.tokens_x.map(lambda row: ' '.join(row))
    gold['snippet_y'] = gold.tokens_y.map(lambda row: ' '.join(row))
    sample = gold[['relation', 'snippet_x', 'snippet_y']]
    negative = read_negative(file.replace('.edus', ''), features=True)
    negative['relation'] = 0
    negative['len_x'] = negative.tokens_x.map(len)
    negative = negative[negative.len_x < MAX_LEN]
    negative['len_y'] = negative.tokens_y.map(len)
    negative = negative[negative.len_y < MAX_LEN]
    negative['snippet_x'] = negative.tokens_x.map(lambda row: ' '.join(row))
    negative['snippet_y'] = negative.tokens_y.map(lambda row: ' '.join(row))
    sample = pd.concat([sample, negative[['relation', 'snippet_x', 'snippet_y']]])
    sample = sample.sort_values(['relation'], ascending=True).drop_duplicates(['snippet_x', 'snippet_y'], keep='last') 
    test_samples.append(sample)

test_samples = pd.concat(test_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
test_samples.reset_index(level=0, inplace=True)
test_samples[['relation', 'snippet_x', 'snippet_y', 'index']].to_csv(TEST_FILE_PATH, sep='\t', header=False, index=False)

###  Generate config file

In [12]:
%%writefile $MODEL_PATH/config.json

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.

{
  "dataset_reader": {
    "type": "quora_paraphrase",
    "lazy": false,
    "token_indexers": {
      "bert": {
          "type": "bert-pretrained",
          "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
          "do_lowercase": false,
          "use_starting_offsets": true
      },
      "token_characters": {
        "type": "characters"
      }
    }
  },
  "train_data_path": "structure_predictor_lstm/structure_cf_train.tsv",
  "validation_data_path": "structure_predictor_lstm/structure_cf_test.tsv",
  "model": {
    "type": "bimpm",
    "dropout": 0.1,
    "text_field_embedder": {
        "allow_unmatched_keys": true,
        "embedder_to_indexer_map": {
            "bert": ["bert", "bert-offsets"],
            "token_characters": ["token_characters"],
        },
        "token_embedders": {
            "bert": {
                "type": "bert-pretrained",
                "pretrained_model": "rubert_cased_L-12_H-768_A-12_pt",
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 20,
                    "padding_index": 0
                },
                "encoder": {
                    "type": "gru",
                    "input_size": 20,
                    "hidden_size": 50,
                    "num_layers": 1,
                    "bidirectional": true
              }
            }
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": 768+100,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 768+100,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": 400,
      "hidden_size": 200,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 200,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": 100,
      "num_layers": 2,
      "dropout": 0.1
    },
    "classifier_feedforward": {
      "input_dim": 400,
      "num_layers": 2,
      "hidden_dims": [200, 2],
      "activations": ["relu", "linear"],
      "dropout": [0.1, 0.0]
    },
    "initializer": [
      [".*linear_layers.*weight", {"type": "xavier_normal"}],
      [".*linear_layers.*bias", {"type": "constant", "val": 0}],
      [".*weight_ih.*", {"type": "xavier_normal"}],
      [".*weight_hh.*", {"type": "orthogonal"}],
      [".*bias.*", {"type": "constant", "val": 0}],
      [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
    ]
  },
  "iterator": {
    "type": "basic",
    "batch_size": 8
  },
  "trainer": {
    "num_epochs": 40,
    "patience": 5,
    "cuda_device": 0,
    "grad_clipping": 5.0,
    "validation_metric": "+accuracy",
    "optimizer": {
      "type": "bert_adam",
      "lr": 0.001
    }
  }
}

Overwriting models/structure_predictor_lstm/config.json
